<a href="https://colab.research.google.com/github/victorefunes/BigDataUDESA/blob/master/Tut2_Twitter_y_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TWITTER Y WEB SCRAPING**


**Profesor**: Walter Sosa Escudero

**Tutora**: Carla Srebot

**Objetivo**: Aprender qué es una API, usarla para bajar tweets y aprender a hacer web scraping usando Beautiful Soup.

---

## Bajando tweets

En el código que sigue veremos cómo bajar tweets de manera sencilla

### Tweepy
Para más información, visitar:  https://readthedocs.org/projects/tweepy/downloads/pdf/latest/

In [ ]:
# Defimos directorio de trabajo
import os  
dir = 'C:\\Users\\csreb\\Dropbox\\Tutoriales\\Big Data 2020\\Tutoriales\\Tutorial 2 - Web Scraping'
os.getcwd()  
os.chdir(dir)  

# Para escribir archivos csv
import csv

# Para bajar tweets: 
# Primero, vamos a necesitar instalar el paquete llamada "tweepy" - esto lo pueden hacer directamente desde la terminal de Spyder
# pip install tweepy
import tweepy

#Credenciales de Twitter
consumer_key = '2MlfpmBC4d8zzz1cZxM1DhxMm'
consumer_secret = 'QxcmeHECRZhGRZ0HqkNglxT5SOAg3y4jRnGaGw6j2fxOmM3f16'
access_key = "1158370347089113088-dTsbvEWhDnKBwSpSXG9MnHHIn7HKHl"
access_secret = "0QxtTRAfl6FLuIuBQxFUPQ3JxdnyBfjuGcZkDUowIYW7S"

#Buscamos los útimos tweets de alguien
def get_tweets(username):
    
    # Twitter requiere que todas las solicitudes utilicen OAuth para la autenticación:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    # Entonces, ahora que tenemos nuestro OAuthHandler equipado con un token de acceso 
    api = tweepy.API(auth)

    #¿Cuántos tweets? - ¡cuidado con el número de tweets!
    number_of_tweets = 100

    #¡Tomemos esos tweets!
    tweets_for_csv = []
    # A continuación, vamos a iterar a través de todos tweets de algún 'user' (¡argumento de la función!)
    for tweet in tweepy.Cursor(api.user_timeline, screen_name = username).items(number_of_tweets):
        #Armo una matriz con la información: 
        #                       usuario       id            fecha                 texto
        tweets_for_csv.append([username, tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")]) 

    # Lo paso a un archivo csv
    outfile = username + "_tweets.csv" # string variable
    print("Escribiendo en " + outfile)
    
    # Vamos a trabajar con el comando 'with' que nos proporciona una sintaxis más limpia para trabajar con objetos de archivo. 
    # Una ventaja de utilizar este método es que los archivos abiertos se cerrarán automáticamente una vez que haya terminado.
    with open(outfile, 'w+') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerows(tweets_for_csv)
        
    # Ojo: 'w+' abre un archivo para escribir y leer. Sobrescribe el archivo existente si el archivo existe. 
    #       Si el archivo no existe, crea un nuevo archivo para leer y escribir.
    

# Vamos a buscar los tweets de los profesores y departamento de Economía:
users = ['wsosaescudero', 'Tommy_E_Murphy', 'EconUdesa']

for user in users:
    get_tweets(user)

### Usando #Hashtags

In [ ]:
import tweepy
import csv

consumer_key = '2MlfpmBC4d8zzz1cZxM1DhxMm'
consumer_secret = 'QxcmeHECRZhGRZ0HqkNglxT5SOAg3y4jRnGaGw6j2fxOmM3f16'
access_token = "1158370347089113088-dTsbvEWhDnKBwSpSXG9MnHHIn7HKHl"
access_token_secret = "0QxtTRAfl6FLuIuBQxFUPQ3JxdnyBfjuGcZkDUowIYW7S"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

csvFile = open('ua.csv', 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,q="#Obama",count=100,
                           lang="en",
                           since="2020-08-12").items():
    print (tweet.created_at, tweet.text)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

### Sentiment Analysis

In [ ]:
# pip install textblob

import textblob
from textblob import TextBlob

# ¿Analizar texto? ¿Qué tipo?

# Cada palabra del léxico tiene puntuaciones para: 
# 1)     polarity: negative vs. positive    (-1.0 => +1.0)
# 2) subjectivity: objective vs. subjective (+0.0 => +1.0)

wiki=TextBlob("Este tutorial es genial")
wiki.tags
wiki.words
wiki.sentiment.polarity

wiki2=TextBlob("This tutorial is awesome") # ¿Qué pasa si agrego 'not'?
wiki2.tags
wiki2.words
wiki2.sentiment.polarity

text = '''The teacher of this class is quite strange. She keeps
telling us that what she's doing is awesome, but I find it rather
boring. I will try to scape and get some coffee at the buffet.
'''

blob = TextBlob(text)
blob.tags       
blob.sentences
                
for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

blob.translate(to="es")  

#### Analizando tweets...

In [ ]:
import tweepy

consumer_key = '2MlfpmBC4d8zzz1cZxM1DhxMm'
consumer_secret = 'QxcmeHECRZhGRZ0HqkNglxT5SOAg3y4jRnGaGw6j2fxOmM3f16'
access_key = "1158370347089113088-dTsbvEWhDnKBwSpSXG9MnHHIn7HKHl"
access_secret = "0QxtTRAfl6FLuIuBQxFUPQ3JxdnyBfjuGcZkDUowIYW7S"

def get_tweets(username):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #¿Cuántos tweets?
    number_of_tweets = 1

    #¡Tomemos esos tweets!
    for tweet in tweepy.Cursor(api.user_timeline, screen_name = username).items(number_of_tweets):
        analysis=TextBlob(tweet.text)
        print(analysis)
        print(analysis.sentiment)
        
users_less_famous = ['realDonaldTrump', 'BarackObama']

for user in users_less_famous:
    get_tweets(user)

### Twitter sin API: Web scraping

Podemos no usar la API

Y esto, queridos alumnos, es hacer web scraping!

In [ ]:
# Para contar el número de tweets de una cuenta en particular:
from bs4 import BeautifulSoup, Comment
import requests

user_agent_phone = 'Mozilla/5.0 (iPhone; CPU iPhone OS 5_1 like Mac OS X) AppleWebKit/534.46 (KHTML, like Gecko) Version/5.1 Mobile/9B179 Safari/7534.48.3'

headers = { 'User-Agent': user_agent_phone}

handle = input('Nombre de la cuenta de Twitter: ')
temp = requests.get('https://twitter.com/'+handle, headers=headers)
bs = BeautifulSoup(temp.text,'lxml')

try:
    tweets = bs.find('td',{'class':'stat'}).find('div',{'class':'statnum'}).text
    print("{} publicó {} tweets y comentarios.".format(handle,tweets))

except:
    print('Nombre de cuenta no encontrado.')

Pero no todo es Twitter en la vida. Podríamos querer bajar precios de supermercados.

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os 
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

lista = ['frutas','verduras']

name_product = []
url_fruits = []
price = []

for i in lista:
    url = "https://www.jumbo.com.ar/" 

    driver = webdriver.Chrome('C:\\Program Files\\ChromeDiver\\chromedriver.exe')
    # Para descargar chromedriver executable entrar al siguiente link: https://sites.google.com/a/chromium.org/chromedriver/home
    driver.get(url)
    time.sleep(1)
    driver.get(url+'frutas-y-verduras/{}'.format(i))

    time.sleep(1)
    elem = driver.find_element_by_tag_name("html")

    no_of_pagedowns = 50
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(.5)
        no_of_pagedowns-=1
        
    html = driver.page_source 
    soup = BeautifulSoup(html) 
     
    data = soup.findAll('li',attrs={'layout':'1579df47-6ea5-4570-a858-8067a35362be',
                                     'class':'frutas-y-verduras-ofertas-y-descuentos-en-frutas-y-verduras-|-jumbo'})
    for elm in data:
        title = elm.find('a',{'class':'product-item__name'}).get('title')
        link = elm.find('a',{'class':'product-item__name'}).get('href')
        try:
            precio = elm.find('span',{'class':'product-prices__value product-prices__value--best-price'}).text
        except:
            precio='None'
         
        name_product.append(title)
        url_fruits.append(link)
        price.append(precio) 
    
    driver.close()
    driver.quit()

fruits_vegsdf = {'nombre':name_product,'precio':price,'link':url_fruits}
df = pd.DataFrame(data=fruits_vegsdf)
writer = pd.ExcelWriter('fruits_vegsdf.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save() 

## EJERCICIOS (Tarea 2)
**Fecha de entrega**: Miércoles 26/08 hasta las 23:59 hs. 

**Reglas de presentación**: 
* Enviar el código de Python por mail a csrebotroeder@udesa.edu.ar con el asunto "Ejercicios tutorial 2 - APELLIDOS". Por cada 5 minutos tarde en la entrega, se restará 1 punto sobre 10.
* El trabajo debe elaborarse en grupos de **tres** personas. Cada grupo debe entregar **un solo trabajo**. 




**EJERCICIO 1:**

Escribir un código que guarde en un csv el usuario, la fecha de creación, el texto y la cantidad de veces que se le dio like a los últimos 75 tweets de Walter.


**EJERCICIO 2:**

* ¿Para qué sirve este código? 
* ¿Qué pondría en el lugar de las X en el print() dentro de try?

In [ ]:
from bs4 import BeautifulSoup, Comment
import requests

user_agent_phone = 'Mozilla/5.0 (iPhone; CPU iPhone OS 5_1 like Mac OS X) AppleWebKit/534.46 (KHTML, like Gecko) Version/5.1 Mobile/9B179 Safari/7534.48.3'

headers = { 'User-Agent': user_agent_phone}

handle = input('Nombre de la cuenta de Twitter: ')
temp = requests.get('https://twitter.com/'+handle, headers=headers)
bs = BeautifulSoup(temp.text,'lxml')

try:
    mat = bs.find_all('div',{'class':'statnum'})
    seg1 = mat[1].text
    seg2 = mat[2].text
    
    var = bs.find('div',{'class':'bio'}).text
    var = var.rstrip()
    var = var.lstrip()

    print('''\n\n{} XXXXX {} XXXXXXXX.
          \n{} XXXXX {} XXXXXXXX.
          \nXX XXXXXXX XX {} XX {}'''.format(handle,seg1, handle, seg2, handle, var))

except:
    print('Nombre de cuenta no encontrado.')

**EJERCICIO 3:**

El siguiente código es una continuación del código de scraping del supermercado Jumbo. 
* Explique **detalladamente** para qué sirve cada línea de código y cuál es la utilidad del mismo.
* ¿Cuáles son los resultados que obtengo?
* ¿Qué representan las listas 'W' y 'Y'?

In [ ]:
file = 'fruits_vegsdf.xlsx'
sheetname="Sheet1"

os.chdir(dir)
cwd = os.getcwd()
cwd

ylist = []
wlist = []

xl = pd.ExcelFile(file)
df = xl.parse(sheetname)

for index, row in df.iterrows():
    urlpage =  row['link'] 
    
    try: 
        page = urllib.request.urlopen(urlpage)
        soup = BeautifulSoup(page, 'html.parser') 
           
        y = soup.find('div',{'class':'skuReference'}).text         
        w = soup.find('span',{'class':'brand'}).text         
    except:
        y = 'None'
        w = 'None'
        
    ylist.append(y)
    wlist.append(w)

df['Y'] = ylist
df['W'] = wlist
df = df[['nombre', 'Y', 'W', 'precio', 'link']]
writer = pd.ExcelWriter('fruit and vegs results.xlsx', engine='xlsxwriter')
df.drop_duplicates(keep='first')  .to_excel(writer, sheet_name='Sheet1', index=False)
writer.save() 